In [ ]:
# EFFR
# https://www.newyorkfed.org/markets/reference-rates/effr

# Settlements FFF
# https://my.apps.factset.com/workstation/navigator/company-security/futures-montage/FF00-USA

In [2]:
from EcoWatch.NelsonSiegelCurvature import NSC
from EcoWatch.Scraping import tbond
import pandas as pd
import numpy as np

In [3]:
tbond_daiy = tbond('2020', '2025')
tbond_daiy = tbond_daiy.drop(['20 Yr', '30 Yr', '1.5 Month'], axis=1)
tbond_daiy = tbond_daiy.interpolate(axis=1)
tbond_daiy = tbond_daiy.dropna(axis=0)

# Define the maturities of the US Treasury Bonds
maturities = np.unique([
    int(col.split()[0]) / 12 if "Mo" in col else int(col.split()[0])
    for col in tbond_daiy.columns
])
# Define the curve maturities
curve_maturities = np.arange(start=maturities.min(), stop=maturities.max()+maturities.min(), step=maturities.min()).round(4)

# Define parameter bounds for optimization: 
bounds = [(0, 1), (-1, 1), (-1, 1), (0, 5)] # Intercept (β0), Slope (β1), Curvature (β2), and Lambda (λ)
x0 = [0.01, 0.01, 0.01, 0.5] # Initial guess for the optimization algorithm
method = 'trust-constr' # Optimization method used for minimization

# Initialisation du modèle
NelsonSiegelCurvature = NSC(maturities=maturities, bounds=bounds, x0=x0, method=method)

# Ajustement des paramètres Nelson-Siegel
nsc_df = NelsonSiegelCurvature.fit(yields=tbond_daiy)

Fitting Nelson-Siegel: 665it [01:01, 12.42it/s]C:\Users\Matteo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\optimize\_differentiable_functions.py:316: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)
Fitting Nelson-Siegel: 791it [01:12, 11.71it/s]C:\Users\Matteo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\optimize\_differentiable_functions.py:316: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self

In [4]:
nsc_df.reset_index().to_csv('nsc.csv', index=False)

In [24]:
buckets = {
    "0.00–0.25": 0.125,
    "0.25–0.50": 0.375,
    "0.50–0.75": 0.625,
    "0.75–1.00": 0.875,
    "1.00–1.25": 1.125,
    "1.25–1.50": 1.375,
    "1.50–1.75": 1.625,
    "1.75–2.00": 1.875,
    "2.00–2.25": 2.125,
    "2.25–2.50": 2.375,
    "2.50–2.75": 2.625,
    "2.75–3.00": 2.875,
    "3.00–3.25": 3.125,
    "3.25–3.50": 3.375,
    "3.50–3.75": 3.625,
    "3.75–4.00": 3.875,
    "4.00–4.25": 4.125,
    "4.25–4.50": 4.375,
    "4.50–4.75": 4.625,
    "4.75–5.00": 4.875,
    "5.00–5.25": 5.125,
    "5.25–5.50": 5.375,
    "5.50–5.75": 5.625,
    "5.75–6.00": 5.875,
    "6.00–6.25": 6.125,
    "6.25–6.50": 6.375,
    "6.50–6.75": 6.625,
    "6.75–7.00": 6.875,
}

In [46]:
buckets = {
    "4.00–4.25": 4.125,
    "4.25–4.50": 4.375,
    "4.50–4.75": 4.625
}

In [47]:
effr = pd.read_csv('overnight_rates.csv', index_col='Effective Date')
effr.index = pd.to_datetime(effr.index, format='mixed')
effr = effr[effr['Rate Type'] == 'EFFR']['Rate (%)']
effr = effr.sort_index()

In [49]:
import numpy as np
from scipy.optimize import minimize

# Buckets (milieux des intervalles en taux annualisé)
bucket_centers = np.array([bucket for bucket in buckets.values()])
n = len(bucket_centers)

# Taux moyen implicite à partir du contrat (ex : prix = 95.7 → 4.3%)
avg_rate = effr.iloc[-1]

# Fonction objectif : on veut minimiser ||p||² → probas "lisses"
def objective(p):
    return np.sum(p**2)

# Contraintes
def probas_sum_to_1(p):
    return np.sum(p) - 1

def mean_matches_future(p):
    return np.dot(p, bucket_centers) - avg_rate

# Dictionnaire de contraintes pour minimize
constraints = [
    {'type': 'eq', 'fun': probas_sum_to_1},
    {'type': 'eq', 'fun': mean_matches_future},
]

# Bornes : toutes les probas entre 0 et 1
bounds = [(0, 1)] * n

# Initial guess : uniform distribution
p0 = np.ones(n) / n

# Optimisation
result = minimize(objective, p0, bounds=bounds, constraints=constraints)

# Résultat
if result.success:
    proba_buckets = {f"{bucket_centers[i]-0.125:.2f}–{bucket_centers[i]+0.125:.2f}": round(p, 4)
                     for i, p in enumerate(result.x)}
    print(proba_buckets)
else:
    print("Optimisation échouée :", result.message)


{'4.00–4.25': np.float64(0.4233), '4.25–4.50': np.float64(0.3333), '4.50–4.75': np.float64(0.2433)}


In [ ]:
#fed_funds = fred(key=fred_key, ticker='FEDFUNDS')/100